In [1]:
import boto3
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from google.cloud import vision
import requests
from nltk.corpus import stopwords as stpword
from nltk.stem import WordNetLemmatizer
import json
from nltk.corpus import stopwords as stpword
from nltk.stem import WordNetLemmatizer
from utilities.image_description_scoring import matching_label_score
from time import time
from utilities.pythonDB import writeToDB, recordsExists

In [2]:
stopwords = set(stpword.words('english'))
wordNetLemma = WordNetLemmatizer()

def extract_clean_list(original_list):
    cleaned = list(set([wordNetLemma.lemmatize(obj_item.lower().strip()) for item in original_list for obj_item in item.split() if obj_item not in stopwords]))
    return [item.strip() for item in [re.sub('[^a-zA-Z0-9]+', ' ', item, flags=re.UNICODE) for item in cleaned]]

def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def get_captions(base_path, annotation_file):
    with open(os.path.join(base_path, annotation_file)) as json_file:
        caption_file = json.load(json_file)
    captions, mapped_annotation = {}, {}

    for annotation in caption_file['annotations']:
        mapped_annotation[annotation['image_id']] = annotation['caption']

    for image in caption_file['images']:
        captions[image['file_name']] = mapped_annotation.get(image['id'])
    return captions

In [3]:
class DetectLabels():
    def __init__(self, base, file, max_labels=30):
        
        with io.open(os.path.join(base, file), 'rb') as image_file:
            content = (image_file.read())
        self.content = content
        self.base, self.file, self.actual_str, self.detected_str = base, file, '', ''
        self.label = ''
        self.start = time()
        self.max_labels = max_labels
        
    def return_function(self, name):
        dataset = self.base.split('/')[1]
        if recordsExists(self.file, dataset, name):
            return
        
        status, score =  getattr(self, 'if_' + name)()
        compute_time = time() - self.start
        self.label = name
        
        bag = (self.file, dataset, self.label, status, score, compute_time, self.actual_str, self.detected_str)
        writeToDB(bag)
        
    def compute_ground_truth(self):
        ''' Status = {-1:'API Error', -2: 'Ground Truth Empty', 0: 'All Correct'}, Return = Ground Truth, Status
        '''
        ground_truth_str = captions.get(self.file)
        
        if ground_truth_str is None or len(ground_truth_str) == 0:
            return '', -2
        else:
            ground_truth_str = extract_clean_list(ground_truth_str.lower().split(' '))
            self.actual_str = ground_truth_str
            return ground_truth_str, 0
    
    def if_gc(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        
        try:
            vision_client = vision.ImageAnnotatorClient()
            image = vision.types.Image(content=self.content)
            text_detection_response = vision_client.label_detection(image=image)
            detected_label = extract_clean_list([str(label.description).lower() for label in text_detection_response.label_annotations])
            score1 = matching_label_score(detected_label, ground_truth, 1)
            score2 = matching_label_score(detected_label, ground_truth, 2)
            score3 = matching_label_score(detected_label, ground_truth, 3)
            score = str([score1, score2, score3])
            
            self.detected_str = detected_label
            return 0, score
        except:
            return -1, 0
            
    def if_aws(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        
        try:
            imgobj = {'Bytes': self.content}
            client=boto3.client('rekognition', region_name='us-east-1')
            response=client.detect_labels(Image=imgobj, MaxLabels=self.max_labels)
            detected_label = [label['Name'] for label in response['Labels']]
            score1 = matching_label_score(detected_label, ground_truth, 1)
            score2 = matching_label_score(detected_label, ground_truth, 2)
            score3 = matching_label_score(detected_label, ground_truth, 3)
            score = str([score1, score2, score3])
            
            self.detected_str = detected_label
            return 0, score
        except:
            return -1, 0
        
    def if_azure(self):
        ground_truth, xml_status = self.compute_ground_truth()
        if xml_status != 0: return xml_status, 0
        
        try:
            subscription_key = os.environ['COMPUTER_VISION_SUBSCRIPTION_KEY']
            endpoint = os.environ['COMPUTER_VISION_ENDPOINT']
            ocr_url = endpoint + "vision/v2.1/describe"
            headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
            params = {'maxCandidates': self.max_labels}
            response = requests.post(ocr_url, headers=headers, params=params, data=self.content).json()
            detected_label = response['description']['tags']
            score1 = matching_label_score(detected_label, ground_truth, 1)
            score2 = matching_label_score(detected_label, ground_truth, 2)
            score3 = matching_label_score(detected_label, ground_truth, 3)
            score = str([score1, score2, score3])
            
            self.detected_str = detected_label
            return 0, score
        except:
            return -1, 0

In [4]:
annotation_dir_path = 'datasets/image_labeling/annotations'
train_dir_path = 'datasets/image_labeling/train2017'
captions_validation = get_captions(annotation_dir_path, 'captions_val2017.json')
captions = get_captions(annotation_dir_path, 'captions_train2017.json')
captions.update(captions_validation)
del captions_validation

In [5]:
dict_files, files_idx = get_mapped(train_dir_path)
shuffled_idx = np.random.randint(0, len(files_idx), len(files_idx))[:10000]

In [ ]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    DetectLabels(train_dir_path, file_name).return_function('azure')

Iterating for File Name = 000000211919.jpg
Iterating for File Name = 000000025541.jpg
Iterating for File Name = 000000328628.jpg
Iterating for File Name = 000000027353.jpg
Iterating for File Name = 000000152871.jpg
Iterating for File Name = 000000390238.jpg
Iterating for File Name = 000000009050.jpg
Iterating for File Name = 000000036281.jpg
Iterating for File Name = 000000367578.jpg
Iterating for File Name = 000000542640.jpg
Iterating for File Name = 000000479712.jpg
Iterating for File Name = 000000121000.jpg
Iterating for File Name = 000000176089.jpg
Iterating for File Name = 000000570418.jpg
Iterating for File Name = 000000276801.jpg
Iterating for File Name = 000000213790.jpg
Iterating for File Name = 000000184157.jpg
Iterating for File Name = 000000419401.jpg
Iterating for File Name = 000000291476.jpg
Iterating for File Name = 000000403693.jpg
Iterating for File Name = 000000502140.jpg
Iterating for File Name = 000000565429.jpg
Iterating for File Name = 000000555805.jpg
Iterating f

Iterating for File Name = 000000355064.jpg
Iterating for File Name = 000000209701.jpg
Iterating for File Name = 000000327146.jpg
Iterating for File Name = 000000115459.jpg
Iterating for File Name = 000000418009.jpg
Iterating for File Name = 000000358429.jpg
Iterating for File Name = 000000223526.jpg
Iterating for File Name = 000000360504.jpg
Iterating for File Name = 000000412613.jpg
Iterating for File Name = 000000153822.jpg
Iterating for File Name = 000000115502.jpg
Iterating for File Name = 000000447439.jpg
Iterating for File Name = 000000466775.jpg
Iterating for File Name = 000000263679.jpg
Iterating for File Name = 000000300930.jpg
Iterating for File Name = 000000555495.jpg
Iterating for File Name = 000000245384.jpg
Iterating for File Name = 000000499266.jpg
Iterating for File Name = 000000350631.jpg
Iterating for File Name = 000000303342.jpg
Iterating for File Name = 000000412873.jpg
Iterating for File Name = 000000106636.jpg
Iterating for File Name = 000000372464.jpg
Iterating f

Iterating for File Name = 000000110819.jpg
Iterating for File Name = 000000124617.jpg
Iterating for File Name = 000000061834.jpg
Iterating for File Name = 000000290566.jpg
Iterating for File Name = 000000091839.jpg
Iterating for File Name = 000000226496.jpg
Iterating for File Name = 000000064516.jpg
Iterating for File Name = 000000023311.jpg
Iterating for File Name = 000000543192.jpg
Iterating for File Name = 000000513811.jpg
Iterating for File Name = 000000513795.jpg
Iterating for File Name = 000000067307.jpg
Iterating for File Name = 000000429283.jpg
Iterating for File Name = 000000246549.jpg
Iterating for File Name = 000000052575.jpg
Iterating for File Name = 000000360698.jpg
Iterating for File Name = 000000442181.jpg
Iterating for File Name = 000000095854.jpg
Iterating for File Name = 000000049625.jpg
Iterating for File Name = 000000192594.jpg
Iterating for File Name = 000000157370.jpg
Iterating for File Name = 000000498555.jpg
Iterating for File Name = 000000064139.jpg
Iterating f

Iterating for File Name = 000000400822.jpg
Iterating for File Name = 000000228388.jpg
Iterating for File Name = 000000149298.jpg
Iterating for File Name = 000000088116.jpg
Iterating for File Name = 000000142000.jpg
Iterating for File Name = 000000302030.jpg
Iterating for File Name = 000000340242.jpg
Iterating for File Name = 000000486936.jpg
Iterating for File Name = 000000426994.jpg
Iterating for File Name = 000000203775.jpg
Iterating for File Name = 000000107481.jpg
Iterating for File Name = 000000373748.jpg
Iterating for File Name = 000000545260.jpg
Iterating for File Name = 000000299468.jpg
Iterating for File Name = 000000559837.jpg
Iterating for File Name = 000000393928.jpg
Iterating for File Name = 000000556524.jpg
Iterating for File Name = 000000084049.jpg
Iterating for File Name = 000000073864.jpg
Iterating for File Name = 000000271338.jpg
Iterating for File Name = 000000410236.jpg
Iterating for File Name = 000000028115.jpg
Iterating for File Name = 000000381228.jpg
Iterating f

Iterating for File Name = 000000064799.jpg
Iterating for File Name = 000000357728.jpg
Iterating for File Name = 000000361073.jpg
Iterating for File Name = 000000122962.jpg
Iterating for File Name = 000000249608.jpg
Iterating for File Name = 000000196876.jpg
Iterating for File Name = 000000400453.jpg
Iterating for File Name = 000000431077.jpg
Iterating for File Name = 000000189015.jpg
Iterating for File Name = 000000154584.jpg
Iterating for File Name = 000000461787.jpg
Iterating for File Name = 000000330727.jpg
Iterating for File Name = 000000490626.jpg
Iterating for File Name = 000000474439.jpg
Iterating for File Name = 000000008176.jpg
Iterating for File Name = 000000242007.jpg
Iterating for File Name = 000000318491.jpg
Iterating for File Name = 000000504606.jpg
Iterating for File Name = 000000507854.jpg
Iterating for File Name = 000000531033.jpg
Iterating for File Name = 000000354718.jpg
Iterating for File Name = 000000491992.jpg
Iterating for File Name = 000000290361.jpg
Iterating f

...

Iterating for File Name = 000000103383.jpg
Iterating for File Name = 000000295831.jpg
Iterating for File Name = 000000511345.jpg
Iterating for File Name = 000000031568.jpg
Iterating for File Name = 000000300929.jpg
Iterating for File Name = 000000423617.jpg
Iterating for File Name = 000000150819.jpg
Iterating for File Name = 000000576119.jpg
Iterating for File Name = 000000256101.jpg
Iterating for File Name = 000000324451.jpg
Iterating for File Name = 000000285963.jpg
Iterating for File Name = 000000299372.jpg
Iterating for File Name = 000000394008.jpg
Iterating for File Name = 000000489470.jpg
Iterating for File Name = 000000474680.jpg
Iterating for File Name = 000000416653.jpg
Iterating for File Name = 000000490284.jpg
Iterating for File Name = 000000048708.jpg
Iterating for File Name = 000000526560.jpg
Iterating for File Name = 000000188151.jpg
Iterating for File Name = 000000059202.jpg
Iterating for File Name = 000000311478.jpg
Iterating for File Name = 000000040143.jpg
Iterating f

Iterating for File Name = 000000055344.jpg
Iterating for File Name = 000000087737.jpg
Iterating for File Name = 000000282283.jpg
Iterating for File Name = 000000384122.jpg
Iterating for File Name = 000000552717.jpg
Iterating for File Name = 000000351253.jpg
Iterating for File Name = 000000164672.jpg
Iterating for File Name = 000000220171.jpg
Iterating for File Name = 000000543549.jpg
Iterating for File Name = 000000160041.jpg
Iterating for File Name = 000000095833.jpg
Iterating for File Name = 000000286939.jpg
Iterating for File Name = 000000555583.jpg
Iterating for File Name = 000000139105.jpg
Iterating for File Name = 000000361790.jpg
Iterating for File Name = 000000432363.jpg
Iterating for File Name = 000000243725.jpg
Iterating for File Name = 000000357076.jpg
Iterating for File Name = 000000321610.jpg
Iterating for File Name = 000000567430.jpg
Iterating for File Name = 000000279071.jpg
Iterating for File Name = 000000112905.jpg
Iterating for File Name = 000000171495.jpg
Iterating f

Iterating for File Name = 000000366068.jpg
Iterating for File Name = 000000552740.jpg
Iterating for File Name = 000000209288.jpg
Iterating for File Name = 000000292554.jpg
Iterating for File Name = 000000331490.jpg
Iterating for File Name = 000000451922.jpg
Iterating for File Name = 000000019927.jpg
Iterating for File Name = 000000493334.jpg
Iterating for File Name = 000000458598.jpg
Iterating for File Name = 000000035245.jpg
Iterating for File Name = 000000567494.jpg
Iterating for File Name = 000000184400.jpg
Iterating for File Name = 000000174511.jpg
Iterating for File Name = 000000006414.jpg
Iterating for File Name = 000000294229.jpg
Iterating for File Name = 000000008074.jpg
Iterating for File Name = 000000410742.jpg
Iterating for File Name = 000000510290.jpg
Iterating for File Name = 000000362007.jpg
Iterating for File Name = 000000562989.jpg
Iterating for File Name = 000000143749.jpg
Iterating for File Name = 000000048129.jpg
Iterating for File Name = 000000513618.jpg
Iterating f

Iterating for File Name = 000000442463.jpg
Iterating for File Name = 000000243091.jpg
Iterating for File Name = 000000002544.jpg
Iterating for File Name = 000000348669.jpg
Iterating for File Name = 000000291788.jpg
Iterating for File Name = 000000132316.jpg
Iterating for File Name = 000000388853.jpg
Iterating for File Name = 000000356261.jpg
Iterating for File Name = 000000055903.jpg
Iterating for File Name = 000000245409.jpg
Iterating for File Name = 000000084615.jpg
Iterating for File Name = 000000044504.jpg
Iterating for File Name = 000000014135.jpg
Iterating for File Name = 000000245915.jpg
Iterating for File Name = 000000333402.jpg
Iterating for File Name = 000000201758.jpg
Iterating for File Name = 000000396150.jpg
Iterating for File Name = 000000553364.jpg
Iterating for File Name = 000000508944.jpg
Iterating for File Name = 000000143436.jpg
Iterating for File Name = 000000444483.jpg
Iterating for File Name = 000000482314.jpg
Iterating for File Name = 000000316605.jpg
Iterating f

Iterating for File Name = 000000411047.jpg
Iterating for File Name = 000000187989.jpg
Iterating for File Name = 000000575343.jpg
Iterating for File Name = 000000317424.jpg
Iterating for File Name = 000000195998.jpg
Iterating for File Name = 000000494403.jpg
Iterating for File Name = 000000406451.jpg
Iterating for File Name = 000000442467.jpg
Iterating for File Name = 000000335037.jpg
Iterating for File Name = 000000046149.jpg
Iterating for File Name = 000000078750.jpg
Iterating for File Name = 000000425035.jpg
Iterating for File Name = 000000437862.jpg
Iterating for File Name = 000000418418.jpg
Iterating for File Name = 000000259983.jpg
Iterating for File Name = 000000117644.jpg
Iterating for File Name = 000000336990.jpg
Iterating for File Name = 000000523988.jpg
Iterating for File Name = 000000544611.jpg
Iterating for File Name = 000000213527.jpg
Iterating for File Name = 000000096894.jpg
Iterating for File Name = 000000471962.jpg
Iterating for File Name = 000000466052.jpg
Iterating f

In [ ]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    DetectLabels(train_dir_path, file_name).return_function('aws')

In [ ]:
for idx in shuffled_idx:
    file_name = dict_files.get(idx)
    print ("Iterating for File Name = {}".format(file_name))
    DetectLabels(train_dir_path, file_name).return_function('gc')